In [1]:
# imports
import pandas as pd
import datetime
from datetime import date
import numpy as np

In [2]:
# load data
customers = pd.read_csv(r'customers_tm1_e.csv')
transactions = pd.read_csv(r'transactions_tm1_e.csv')

## 1 Brief Look at the data

In [3]:
print(customers.shape)
customers.head()

(116016, 5)


,customer_id,dob,state,start_balance,creation_date
0,91,1993-07-01,California,10180.56,2007-01-31
1,92,1985-12-05,New York,4757.68,2007-01-31
2,93,1987-11-19,Minnesota,6796.72,2007-01-31
3,94,1981-03-23,Minnesota,9870.48,2007-01-31
4,95,1970-04-06,California,12500.72,2007-01-31


In [4]:
print(transactions.shape)
transactions.head()

(4977972, 7)


,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal
0,2007-01-31,24137947,91,3034.26,2007-01-31,3034.26,0.00
1,2007-01-31,24137947,91,-5295.18,2007-01-16,0.00,-5295.18
2,2007-02-28,24137947,91,0.00,2007-02-28,0.00,0.00
3,2007-03-31,24137947,91,-0.00,2007-03-30,0.00,-0.00
4,2007-03-31,24137947,91,-0.00,2007-03-11,0.00,-0.00


In [5]:
transactions[transactions['customer_id']==607]

,date,account_id,customer_id,amount,transaction_date,deposit,withdrawal
29048,2007-01-31,24138463,607,2438.56,2007-01-31,2438.56,0.00
29049,2007-02-28,24138463,607,2358.10,2007-02-28,2358.10,0.00
29050,2007-02-28,24138463,607,-3670.89,2007-02-24,0.00,-3670.89
29051,2007-03-31,24138463,607,2759.53,2007-03-31,2759.53,0.00
29052,2007-04-30,24138463,607,0.00,2007-04-30,0.00,0.00
...,...,...,...,...,...,...,...
29431,2020-05-31,24138463,607,2663.34,2020-05-31,2663.34,0.00
29432,2020-05-31,24138463,607,-217.08,2020-05-30,0.00,-217.08
29433,2020-05-31,24138463,607,-54.18,2020-05-05,0.00,-54.18
29434,2020-05-31,24138463,607,-180.62,2020-05-22,0.00,-180.62


## 2 Creating the Combined Dataset

#### 2.1 Dataframe of Useful Variables

In [6]:
# create new dataset containing relevant columns
df = customers[['customer_id','dob','state','start_balance','creation_date']]

# adding some columns to new dataset

# final transaction date
transactions['transaction_date'] = pd.to_datetime(transactions['transaction_date']) # convert datetime
last_transaction = transactions.groupby('customer_id', as_index=False)['transaction_date'].max() # create table of last transaction and customer id
last_transaction.rename(columns={'transaction_date':'final_transaction_date'},inplace=True) # rename
df = df.merge(last_transaction, how='left', on='customer_id') # merge to new df

# first transaction date
first_transaction = transactions.groupby('customer_id', as_index=False)['transaction_date'].min()
first_transaction.rename(columns={'transaction_date':'first_transaction_date'},inplace=True)
df = df.merge(first_transaction, how='left', on='customer_id')

# total deposits
tot_deposits = transactions.groupby('customer_id', as_index=False)['deposit'].sum()
tot_deposits.rename(columns={'deposit':'total_deposits'}, inplace=True)
df = df.merge(tot_deposits, how='left', on='customer_id')

# total withdrawals
tot_withdraws = transactions.groupby('customer_id', as_index=False)['withdrawal'].sum()
tot_withdraws.rename(columns={'withdrawal':'total_withdrawals'}, inplace=True)
df = df.merge(tot_withdraws, how='left', on='customer_id')
df['total_withdrawals'] = df['total_withdrawals'] * -1

# final balance
df['final_balance'] = df['start_balance'] + df['total_deposits'] - df['total_withdrawals']

# duration open
df['creation_date'] = pd.to_datetime(df['creation_date'])
df['duration_open'] = (df['final_transaction_date'] - df['first_transaction_date'])
df['duration_open'] = df['duration_open'].dt.days

# age on final transaction date
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = (((df['final_transaction_date'] - df['dob']).dt.days)/365).apply(np.floor)

# avg deposits
avg_deposits = transactions.groupby('customer_id', as_index=False)['deposit'].mean()
avg_deposits.rename(columns={'deposit':'avg_deposit_val'}, inplace=True)
df = df.merge(avg_deposits, how='left', on='customer_id')

# avg withdrawals
avg_withdrawals = transactions.groupby('customer_id', as_index=False)['withdrawal'].mean()
avg_withdrawals.rename(columns={'withdrawal':'avg_withdrawal_val'}, inplace=True)
df = df.merge(avg_withdrawals, how='left', on='customer_id')

# number of deposits and withdrawals
transactions['deposit_with_nas'] = transactions['deposit'].replace({0:np.nan})
transactions['withdrawal_with_nas'] = transactions['withdrawal'].replace({0:np.nan})
new_df = transactions[['customer_id','deposit_with_nas','withdrawal_with_nas']]
df = df.merge(new_df.groupby('customer_id')['deposit_with_nas'].agg('count'), how='left', on='customer_id')
df = df.merge(new_df.groupby('customer_id')['withdrawal_with_nas'].agg('count'), how='left', on='customer_id')
df.rename(columns={'deposit_with_nas':'num_deposits','withdrawal_with_nas':'num_withdrawals'}, inplace=True)

# regions
state_groups = {'Northeast': ['New York', 'Pennsylvania', 'New Jersey', 'Connecticut', 'Massachusetts', 'Rhode Island', 'Maine', 'Vermont', 'New Hampshire'],
                'Midwest': ['Illinois', 'Ohio', 'Michigan', 'Indiana', 'Wisconsin', 'Minnesota', 'Iowa', 'Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas'],
                'South': ['Texas', 'Florida', 'North Carolina', 'Georgia', 'Virginia', 'Tennessee', 'South Carolina', 'Alabama', 'Louisiana', 'Kentucky', 'Oklahoma', 'Arkansas', 'West Virginia', 'Mississippi'],
                'West': ['California', 'Washington', 'Arizona', 'Colorado', 'Oregon', 'Utah', 'Nevada', 'New Mexico', 'Idaho', 'Montana', 'Wyoming', 'Alaska', 'Hawaii', 'District of Columbia', 'Delaware']}
state_to_region = {}
for region, states in state_groups.items():
    for state in states:
        state_to_region[state] = region
# Apply the mapping to the 'state' column to create a new 'Region' column
df["region"] = df['state'].apply(lambda x: state_to_region[x] if x in state_to_region else 'Other')

## 2.2 Adding 'Exited' Target Variable

In [7]:
# create exited target variable
end_date = pd.to_datetime('2020-5-31')
exited = [(1 if row < end_date else 0) for row in df['final_transaction_date']]
# (df['churn'] == 1) if (last_customer_date < end_date) else 0
df['exited'] = exited
df['exited'].value_counts()

1    100163
0     15853
Name: exited, dtype: int64

In [8]:
df.head()

,customer_id,dob,state,start_balance,creation_date,final_transaction_date,first_transaction_date,total_deposits,total_withdrawals,final_balance,duration_open,age,avg_deposit_val,avg_withdrawal_val,num_deposits,num_withdrawals,region,exited
0,91,1993-07-01,California,10180.56,2007-01-31,2007-03-30,2007-01-16,3034.26,5295.18,7919.64,73,13.0,505.710000,-882.530000,1,1,West,1
1,92,1985-12-05,New York,4757.68,2007-01-31,2008-03-14,2007-01-31,13292.09,7629.15,10420.62,408,22.0,428.777097,-246.101613,10,16,Northeast,1
2,93,1987-11-19,Minnesota,6796.72,2007-01-31,2007-11-30,2007-01-31,19822.61,18965.15,7654.18,303,20.0,683.538276,-653.970690,10,19,Midwest,1
3,94,1981-03-23,Minnesota,9870.48,2007-01-31,2007-10-31,2007-01-01,17804.39,16791.91,10882.96,303,26.0,539.526970,-508.845758,6,18,Midwest,1
4,95,1970-04-06,California,12500.72,2007-01-31,2008-08-31,2007-01-31,61332.12,48886.87,24945.97,578,38.0,1304.938723,-1040.146170,17,26,West,1


## 3 Cleaning

#### 3.1 Drop Na's

In [9]:
df.isna().sum()
df.dropna(inplace=True)

#### 3.2 Fix States

In [10]:
# fixing states
df['state'].replace(to_replace='TX', value='Texas', inplace=True)
df['state'].replace(to_replace='CALIFORNIA', value='California', inplace=True)
df['state'].replace(to_replace='MASS', value='Massachusetts', inplace=True)
df['state'].replace(to_replace='NY', value='New York', inplace=True)
# drop unidentifiable states
df = df[(df.state != '-999') & (df.state != 'UNK') & (df.state != 'Australia')]

#### 3.3 Drop Useless Columns

In [11]:
cols_to_drop = ['dob','creation_date']
df.drop(columns=cols_to_drop, inplace=True)

# ANY OTHER CLEANING?

#### 3.4 Save Cleaned Dataset

In [13]:
df.to_csv(r'cleaned_data.csv', index=False)

SNS PAIRPLOT

In [ ]:
import seaborn as sns 

sns.pairplot(df)
